In [8]:
import os
import pandas as pd

# ======================
# CẤU HÌNH
# ======================

CSV_PATH = "dataset_processed/CUSTOM2_list.csv"    # CSV cần đồng bộ
IMAGE_ROOT = "dataset_processed"                   # thư mục gốc chứa ảnh

# ======================
# ĐỌC CSV CŨ
# ======================

df = pd.read_csv(CSV_PATH)
print("Số dòng CSV ban đầu:", len(df))

# ======================
# KIỂM TRA FILE EXIST
# ======================

valid_rows = []
missing = 0

for idx, row in df.iterrows():
    img_rel_path = row["path"]                     # ví dụ: CUSTOM2/live/img_01.jpg
    img_full_path = os.path.join(IMAGE_ROOT, img_rel_path)

    if os.path.exists(img_full_path):
        valid_rows.append(row)
    else:
        missing += 1

print("Ảnh bị thiếu (đã xóa):", missing)

# ======================
# TẠO CSV MỚI
# ======================

df_new = pd.DataFrame(valid_rows)
df_new.to_csv(CSV_PATH.replace(".csv", "_clean.csv"), index=False)

print("CSV mới đã lưu:", CSV_PATH.replace(".csv", "_clean.csv"))
print("Số dòng CSV sau khi clean:", len(df_new))


Số dòng CSV ban đầu: 4996
Ảnh bị thiếu (đã xóa): 0
CSV mới đã lưu: dataset_processed/CUSTOM2_list_clean.csv
Số dòng CSV sau khi clean: 4996


In [12]:
import os
import pandas as pd

# ======================
# CẤU HÌNH
# ======================

DOMAIN = "CUSTOM"   # tên domain cần xử lý
CSV_PATH = f"dataset_processed/{DOMAIN}_list.csv"
IMAGE_ROOT = "dataset_processed"      # thư mục gốc chứa các domain

# ======================
# ĐỌC CSV HIỆN TẠI
# ======================

df = pd.read_csv(CSV_PATH)
csv_paths = set(df["path"].tolist())

print("Số ảnh trong CSV hiện tại:", len(csv_paths))

# ======================
# QUÉT TOÀN BỘ ẢNH HIỆN CÓ TRONG DOMAIN
# ======================

domain_dir = os.path.join(IMAGE_ROOT, DOMAIN)
all_images = []

for root, dirs, files in os.walk(domain_dir):
    for f in files:
        if f.lower().endswith((".jpg", ".png", ".jpeg")):
            rel = os.path.relpath(os.path.join(root, f), IMAGE_ROOT)
            rel = rel.replace("\\", "/")   # chuẩn hóa path dạng UNIX
            all_images.append(rel)

print("Tổng ảnh trong thư mục:", len(all_images))

# ======================
# TÌM ẢNH THIẾU TRONG CSV
# ======================

missing_images = [img for img in all_images if img not in csv_paths]

print("Ảnh tồn tại trong thư mục nhưng chưa có trong CSV:", len(missing_images))

# ======================
# TỰ ĐỘNG BỔ SUNG VÀO CSV
# ======================

new_rows = []

for img_path in missing_images:
    # xác định label dựa theo thư mục (live/spoof)
    if "/live/" in img_path.lower():
        label = 1
    elif "/spoof/" in img_path.lower() or "/attack/" in img_path.lower():
        label = 0
    else:
        print("⚠ Không xác định được label cho:", img_path)
        continue

    new_rows.append([img_path, label, DOMAIN])

# nếu có ảnh mới → thêm vào dataframe
if new_rows:
    df_new = pd.concat([df, pd.DataFrame(new_rows, columns=["path", "label", "domain"])], ignore_index=True)
    df_new.to_csv(CSV_PATH.replace(".csv", "_updated.csv"), index=False)
    print("\n✓ CSV đã được cập nhật:", CSV_PATH.replace(".csv", "_updated.csv"))
else:
    print("\nKhông có ảnh nào cần bổ sung vào CSV.")
    

# ======================
# IN RA DANH SÁCH ẢNH ĐƯỢC THÊM
# ======================

if new_rows:
    print("\n===== ẢNH MỚI BỔ SUNG VÀO CSV =====")
    for row in new_rows:
        print(row[0])


Số ảnh trong CSV hiện tại: 6710
Tổng ảnh trong thư mục: 6710
Ảnh tồn tại trong thư mục nhưng chưa có trong CSV: 0

Không có ảnh nào cần bổ sung vào CSV.


In [8]:
import pandas as pd

# ============================
# CẤU HÌNH
# ============================

CSV_PATH = "dataset_processed/list_filtered.csv"   # đường dẫn file CSV cần sửa
PREFIX = "CUSTOM/"                                # chuỗi muốn thêm đầu mỗi path

# ============================
# ĐỌC CSV
# ============================

df = pd.read_csv(CSV_PATH)
print("Số dòng CSV:", len(df))

# ============================
# THÊM PREFIX VÀO CỘT PATH
# ============================

df["path"] = PREFIX + df["path"].astype(str)

# ============================
# LƯU FILE MỚI
# ============================

OUTPUT_PATH = CSV_PATH
df.to_csv(OUTPUT_PATH, index=False)

print("✓ Đã thêm prefix thành công!")
print("→ File mới:", OUTPUT_PATH)


Số dòng CSV: 6710
✓ Đã thêm prefix thành công!
→ File mới: dataset_processed/list_filtered.csv


In [9]:
import pandas as pd

# ============================
# CẤU HÌNH
# ============================

CSV_PATH = "dataset_processed/list_filtered.csv"   # file csv cần sửa
SUFFIX = ",CUSTOM"                               # chuỗi hậu tố muốn thêm

# ============================
# ĐỌC FILE CSV DƯỚI DẠNG TEXT
# ============================

with open(CSV_PATH, "r", encoding="utf-8") as f:
    lines = f.readlines()

# ============================
# THÊM HẬU TỐ VÀO MỖI DÒNG
# ============================

new_lines = []
for line in lines:
    line = line.rstrip("\n")        # bỏ newline cũ
    new_lines.append(line + SUFFIX + "\n")   # thêm hậu tố + dòng mới

# ============================
# LƯU FILE MỚI
# ============================

OUTPUT_PATH = CSV_PATH

with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    f.writelines(new_lines)

print("✓ Đã thêm hậu tố vào CSV")
print("→ File mới:", OUTPUT_PATH)
print("→ Hậu tố thêm:", SUFFIX)


✓ Đã thêm hậu tố vào CSV
→ File mới: dataset_processed/list_filtered.csv
→ Hậu tố thêm: ,CUSTOM


In [ ]:
import os
from PIL import Image

# ============================
# CẤU HÌNH
# ============================

DOMAIN = "CUSTOM3"
ROOT_DIR = "dataset_processed"
TARGET_SIZE = (256, 256)

IMAGE_DIRS = [
    f"{ROOT_DIR}/{DOMAIN}/live",
    f"{ROOT_DIR}/{DOMAIN}/not_live"
]

# ============================
# RESIZE ẢNH
# ============================

total = 0
failed = 0

for img_dir in IMAGE_DIRS:
    if not os.path.exists(img_dir):
        print(f"⚠️ Không tồn tại thư mục: {img_dir}")
        continue

    for fname in os.listdir(img_dir):
        if fname.lower().endswith((".jpg", ".jpeg", ".png")):
            img_path = os.path.join(img_dir, fname)
            try:
                with Image.open(img_path) as img:
                    img = img.convert("RGB")   # đảm bảo 3 channel
                    img = img.resize(TARGET_SIZE, Image.BILINEAR)
                    img.save(img_path)
                    total += 1
            except Exception as e:
                failed += 1
                print(f"❌ Lỗi ảnh: {img_path} | {e}")

# ============================
# KẾT QUẢ
# ============================

print("✓ Resize hoàn tất!")
print("→ Tổng ảnh resize:", total)
print("→ Ảnh lỗi:", failed)
print("→ Kích thước mới:", TARGET_SIZE)


✓ Resize hoàn tất!
→ Tổng ảnh resize: 18827
→ Ảnh lỗi: 0
→ Kích thước mới: (256, 256)


In [4]:
import os
import pandas as pd

# ============================
# CẤU HÌNH
# ============================

DOMAIN = "CUSTOM3"
ROOT_DIR = "dataset_processed"                   # thư mục gốc dataset_processed
LIVE_DIR = f"{ROOT_DIR}/{DOMAIN}/live"
SPOOF_DIR = f"{ROOT_DIR}/{DOMAIN}/not_live"
OUTPUT_CSV = f"{ROOT_DIR}/{DOMAIN}_list.csv"

# ============================
# THU THẬP ẢNH
# ============================

records = []

# LIVE = label 1
if os.path.exists(LIVE_DIR):
    for fname in os.listdir(LIVE_DIR):
        if fname.lower().endswith((".jpg", ".png", ".jpeg")):
            rel_path = f"{DOMAIN}/live/{fname}"
            records.append([rel_path, 1, DOMAIN])

# NOT_LIVE = label 0
if os.path.exists(SPOOF_DIR):
    for fname in os.listdir(SPOOF_DIR):
        if fname.lower().endswith((".jpg", ".png", ".jpeg")):
            rel_path = f"{DOMAIN}/not_live/{fname}"
            records.append([rel_path, 0, DOMAIN])

# ============================
# LƯU CSV
# ============================

df = pd.DataFrame(records, columns=["path", "label", "domain"])
df.to_csv(OUTPUT_CSV, index=False)

print("✓ CSV đã được tạo thành công!")
print("→ Lưu tại:", OUTPUT_CSV)
print("→ Tổng số dòng:", len(df))


✓ CSV đã được tạo thành công!
→ Lưu tại: dataset_processed/CUSTOM3_list.csv
→ Tổng số dòng: 18827


In [5]:
import os
import pandas as pd

# ============================
# CẤU HÌNH
# ============================

ROOT_DIR = "dataset_processed"
OUTPUT_CSV = f"{ROOT_DIR}/list_filtered.csv"

# ============================
# TÌM TẤT CẢ FILE *_list.csv
# ============================

csv_files = [
    os.path.join(ROOT_DIR, f)
    for f in os.listdir(ROOT_DIR)
    if f.endswith("_list.csv")
]

print("Các file CSV được tìm thấy:")
for f in csv_files:
    print("  →", f)

# ============================
# GỘP CSV
# ============================

df_list = []
for f in csv_files:
    df = pd.read_csv(f)
    df_list.append(df)

combined = pd.concat(df_list, ignore_index=True)

print("\nTổng số dòng sau khi gộp:", len(combined))

# ============================
# LOẠI TRÙNG (nếu có)
# ============================

before = len(combined)
combined.drop_duplicates(inplace=True)
after = len(combined)

print(f"Đã loại {before - after} dòng trùng lặp.")

# ============================
# LƯU FILE TỔNG
# ============================

combined.to_csv(OUTPUT_CSV, index=False)
print("\n✓ Hoàn tất! CSV tổng đã lưu tại:", OUTPUT_CSV)

# ============================
# THỐNG KÊ THEO DOMAIN
# ============================

print("\n===== Thống kê theo domain =====")
print(combined["domain"].value_counts())


Các file CSV được tìm thấy:
  → dataset_processed\CUSTOM2_list.csv
  → dataset_processed\CUSTOM3_list.csv
  → dataset_processed\CUSTOM_list.csv
  → dataset_processed\MSU_MFSD_list.csv
  → dataset_processed\SiW_list.csv

Tổng số dòng sau khi gộp: 40097
Đã loại 0 dòng trùng lặp.

✓ Hoàn tất! CSV tổng đã lưu tại: dataset_processed/list_filtered.csv

===== Thống kê theo domain =====
domain
CUSTOM3     18827
SiW          7586
CUSTOM       6710
CUSTOM2      4996
MSU_MFSD     1978
Name: count, dtype: int64
